In [1]:
# 기본 라이브러리
import os
import sys
import numpy as np
import pandas as pd
import torch
import re
from tqdm import tqdm

# 데이터 처리 관련
from sklearn.preprocessing import MultiLabelBinarizer
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torch.nn.functional as F

# Hugging Face Transformers
from transformers import (
    BertConfig, AutoTokenizer, BertForSequenceClassification, AdamW
)

# 평가 관련 (F1 Score, Classification Report)
from seqeval.metrics import f1_score, classification_report

# Config 불러오기
import config

# 라벨 변환 관련 모듈
import label

# 데이터셋 관련 모듈
import Dataset

sys.path.append('/home/lhch9550/공모전/KPF-BERT-CLS')

/home/lhch9550/anaconda3/lib/python3.8/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/lhch9550/anaconda3/lib/python3.8/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [2]:
#시범적으로 22년도 1~3월 기사만을 활용하였음 -> 1백만 건
data1 = pd.read_csv("/home/lhch9550/공모전/KPF-BERT-CLS/NEWSPAPER_2022_1.csv" )
data2 = pd.read_csv("/home/lhch9550/공모전/KPF-BERT-CLS/NEWSPAPER_2022_2.csv" )
data3 = pd.read_csv("/home/lhch9550/공모전/KPF-BERT-CLS/NEWSPAPER_2022_3.csv" )
data3 

,file_id,doc_id,title,author,publisher,date,topic,original_topic,sentence_id,sentence
0,NLRW2300000006,NLRW2300000006.22800,부산일보 2022년 기사,류영신,부산일보,20220928,생활,"지역>경북, 지역>대구, 지역>울산",NLRW2300000006.22800.4,거창군과 서울우유는 코로나19가 조금씩 소극적인 점을 감안해 2023년부터 산업관광...
1,NLRW2300000006,NLRW2300000006.22800,부산일보 2022년 기사,류영신,부산일보,20220928,생활,"지역>경북, 지역>대구, 지역>울산",NLRW2300000006.22800.5,또한 구 군수는 재개관한 서울우유 거창공장 홍보관을 견학하고 산업관광의 재개에 뜻을...
2,NLRW2300000006,NLRW2300000006.22800,부산일보 2022년 기사,류영신,부산일보,20220928,생활,"지역>경북, 지역>대구, 지역>울산",NLRW2300000006.22800.6,산업관광은 서울우유와 거창군이 MOU를 체결해 진행한다. 전 국민을 대상으로 접수해...
3,NLRW2300000006,NLRW2300000006.22800,부산일보 2022년 기사,류영신,부산일보,20220928,생활,"지역>경북, 지역>대구, 지역>울산",NLRW2300000006.22800.7,또한 산업관광을 통해 기업의 이미지를 제고하고 거창군과 서울우유 협동조합이 상생하는...
4,NLRW2300000006,NLRW2300000006.22800,부산일보 2022년 기사,류영신,부산일보,20220928,생활,"지역>경북, 지역>대구, 지역>울산",NLRW2300000006.22800.8,거창군은 산업관광 재개가 지역경제 활성화에도 큰 도움이 될 것으로 내다보고 있다. ...
...,...,...,...,...,...,...,...,...,...,...
999995,NPRW2300000001,NPRW2300000001.31919,머니투데이 2022년 기사,정현수|기자|,머니투데이,20220628,IT/과학,"IT_과학>모바일, IT_과학>콘텐츠, 지역>경기",NPRW2300000001.31919.5,모바일 주민등록증을 이용하려면 본인 명의의 스마트폰으로 서비스 등록을 해야 한다. ...
999996,NPRW2300000001,NPRW2300000001.31919,머니투데이 2022년 기사,정현수|기자|,머니투데이,20220628,IT/과학,"IT_과학>모바일, IT_과학>콘텐츠, 지역>경기",NPRW2300000001.31919.6,이 과정에서 지문이나 얼굴 등 생체인증 정보를 추가로 등록할 수 있다. 신분확인의 ...
999997,NPRW2300000001,NPRW2300000001.31919,머니투데이 2022년 기사,정현수|기자|,머니투데이,20220628,IT/과학,"IT_과학>모바일, IT_과학>콘텐츠, 지역>경기",NPRW2300000001.31919.7,모바일 주민등록증은 개인정보 보호 장치도 마련했다. 기본 화면에는 성명과 주소의 일...
999998,NPRW2300000001,NPRW2300000001.31919,머니투데이 2022년 기사,정현수|기자|,머니투데이,20220628,IT/과학,"IT_과학>모바일, IT_과학>콘텐츠, 지역>경기",NPRW2300000001.31919.8,한창섭 행안부 차관은 “디지털 신원증명으로써 주민등록증 모바일 확인서비스가 시행되면...


In [3]:
# 3개의 데이터셋에 동일한 변환 적용
for dataset in [data1, data2, data3]:
    dataset['NEWS_SML_SUBJ_CD'] = dataset['original_topic'].astype(str).apply(
        lambda x: ', '.join([topic.strip() for topic in x.split('>')[1:]] if '>' in x else '')
    )

# 3개의 데이터셋을 하나로 병합
merged_data = pd.concat([data1, data2, data3], ignore_index=True)
merged_data

,file_id,doc_id,title,author,publisher,date,topic,original_topic,sentence_id,sentence,NEWS_SML_SUBJ_CD
0,NIRW2300000001,NIRW2300000001.1,노컷뉴스 2022년 기사,박중석 부산CBS 박중석 기자,노컷뉴스,20220101,정치,"지역>부산, 지역>경남, 지역>울산",NIRW2300000001.1.1,[영상]“위기를 이겨내고 일상으로” 부산 기관장 신년사,"부산, 지역, 경남, 지역, 울산"
1,NIRW2300000001,NIRW2300000001.1,노컷뉴스 2022년 기사,박중석 부산CBS 박중석 기자,노컷뉴스,20220101,정치,"지역>부산, 지역>경남, 지역>울산",NIRW2300000001.1.2,취임 2년 차를 맞는 박형준 부산시장은 코로나19 위기를 이겨내고 있는 부산시민들에...,"부산, 지역, 경남, 지역, 울산"
2,NIRW2300000001,NIRW2300000001.1,노컷뉴스 2022년 기사,박중석 부산CBS 박중석 기자,노컷뉴스,20220101,정치,"지역>부산, 지역>경남, 지역>울산",NIRW2300000001.1.3,"박 시장은 “코로나19는 번번이 우리의 희망의 길목을 막아섰고, 지금도 여전히 민생...","부산, 지역, 경남, 지역, 울산"
3,NIRW2300000001,NIRW2300000001.1,노컷뉴스 2022년 기사,박중석 부산CBS 박중석 기자,노컷뉴스,20220101,정치,"지역>부산, 지역>경남, 지역>울산",NIRW2300000001.1.4,그러면서 “숱한 어려움을 헤쳐온 위대한 시민의 힘으로 2022년을 코로나를 극복하고...,"부산, 지역, 경남, 지역, 울산"
4,NIRW2300000001,NIRW2300000001.1,노컷뉴스 2022년 기사,박중석 부산CBS 박중석 기자,노컷뉴스,20220101,정치,"지역>부산, 지역>경남, 지역>울산",NIRW2300000001.1.5,박 시장은 “치밀한 전략으로 2030세계박람회 유치를 위한 국제박람회기구 현지 실사...,"부산, 지역, 경남, 지역, 울산"
...,...,...,...,...,...,...,...,...,...,...,...
2999995,NPRW2300000001,NPRW2300000001.31919,머니투데이 2022년 기사,정현수|기자|,머니투데이,20220628,IT/과학,"IT_과학>모바일, IT_과학>콘텐츠, 지역>경기",NPRW2300000001.31919.5,모바일 주민등록증을 이용하려면 본인 명의의 스마트폰으로 서비스 등록을 해야 한다. ...,"모바일, IT_과학, 콘텐츠, 지역, 경기"
2999996,NPRW2300000001,NPRW2300000001.31919,머니투데이 2022년 기사,정현수|기자|,머니투데이,20220628,IT/과학,"IT_과학>모바일, IT_과학>콘텐츠, 지역>경기",NPRW2300000001.31919.6,이 과정에서 지문이나 얼굴 등 생체인증 정보를 추가로 등록할 수 있다. 신분확인의 ...,"모바일, IT_과학, 콘텐츠, 지역, 경기"
2999997,NPRW2300000001,NPRW2300000001.31919,머니투데이 2022년 기사,정현수|기자|,머니투데이,20220628,IT/과학,"IT_과학>모바일, IT_과학>콘텐츠, 지역>경기",NPRW2300000001.31919.7,모바일 주민등록증은 개인정보 보호 장치도 마련했다. 기본 화면에는 성명과 주소의 일...,"모바일, IT_과학, 콘텐츠, 지역, 경기"
2999998,NPRW2300000001,NPRW2300000001.31919,머니투데이 2022년 기사,정현수|기자|,머니투데이,20220628,IT/과학,"IT_과학>모바일, IT_과학>콘텐츠, 지역>경기",NPRW2300000001.31919.8,한창섭 행안부 차관은 “디지털 신원증명으로써 주민등록증 모바일 확인서비스가 시행되면...,"모바일, IT_과학, 콘텐츠, 지역, 경기"


In [4]:
# 전처리 (텍스트 클리닝)
def clean_text(text):
    import re
    text = re.sub(r"([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z-.]+)", "", text)
    text = re.sub(r"[\r\n\xa0]", "", text)
    text = re.sub(r"(\.\s+[ㄱ-ㅎ가-힣]+\s[기]+[자]+)", "", text)
    text = re.sub(r"[^\w\s^.]", " ", text)
    return text

merged_data['NEWS_CNTS'] = merged_data['sentence'].apply(clean_text)
merged_data = merged_data.drop_duplicates(subset=['NEWS_CNTS'], keep='first')
merged_data

,file_id,doc_id,title,author,publisher,date,topic,original_topic,sentence_id,sentence,NEWS_SML_SUBJ_CD,NEWS_CNTS
0,NIRW2300000001,NIRW2300000001.1,노컷뉴스 2022년 기사,박중석 부산CBS 박중석 기자,노컷뉴스,20220101,정치,"지역>부산, 지역>경남, 지역>울산",NIRW2300000001.1.1,[영상]“위기를 이겨내고 일상으로” 부산 기관장 신년사,"부산, 지역, 경남, 지역, 울산",영상 위기를 이겨내고 일상으로 부산 기관장 신년사
1,NIRW2300000001,NIRW2300000001.1,노컷뉴스 2022년 기사,박중석 부산CBS 박중석 기자,노컷뉴스,20220101,정치,"지역>부산, 지역>경남, 지역>울산",NIRW2300000001.1.2,취임 2년 차를 맞는 박형준 부산시장은 코로나19 위기를 이겨내고 있는 부산시민들에...,"부산, 지역, 경남, 지역, 울산",취임 2년 차를 맞는 박형준 부산시장은 코로나19 위기를 이겨내고 있는 부산시민들에...
2,NIRW2300000001,NIRW2300000001.1,노컷뉴스 2022년 기사,박중석 부산CBS 박중석 기자,노컷뉴스,20220101,정치,"지역>부산, 지역>경남, 지역>울산",NIRW2300000001.1.3,"박 시장은 “코로나19는 번번이 우리의 희망의 길목을 막아섰고, 지금도 여전히 민생...","부산, 지역, 경남, 지역, 울산",박 시장은 코로나19는 번번이 우리의 희망의 길목을 막아섰고 지금도 여전히 민생...
3,NIRW2300000001,NIRW2300000001.1,노컷뉴스 2022년 기사,박중석 부산CBS 박중석 기자,노컷뉴스,20220101,정치,"지역>부산, 지역>경남, 지역>울산",NIRW2300000001.1.4,그러면서 “숱한 어려움을 헤쳐온 위대한 시민의 힘으로 2022년을 코로나를 극복하고...,"부산, 지역, 경남, 지역, 울산",그러면서 숱한 어려움을 헤쳐온 위대한 시민의 힘으로 2022년을 코로나를 극복하고...
4,NIRW2300000001,NIRW2300000001.1,노컷뉴스 2022년 기사,박중석 부산CBS 박중석 기자,노컷뉴스,20220101,정치,"지역>부산, 지역>경남, 지역>울산",NIRW2300000001.1.5,박 시장은 “치밀한 전략으로 2030세계박람회 유치를 위한 국제박람회기구 현지 실사...,"부산, 지역, 경남, 지역, 울산",박 시장은 치밀한 전략으로 2030세계박람회 유치를 위한 국제박람회기구 현지 실사...
...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,NPRW2300000001,NPRW2300000001.31919,머니투데이 2022년 기사,정현수|기자|,머니투데이,20220628,IT/과학,"IT_과학>모바일, IT_과학>콘텐츠, 지역>경기",NPRW2300000001.31919.5,모바일 주민등록증을 이용하려면 본인 명의의 스마트폰으로 서비스 등록을 해야 한다. ...,"모바일, IT_과학, 콘텐츠, 지역, 경기",모바일 주민등록증을 이용하려면 본인 명의의 스마트폰으로 서비스 등록을 해야 한다. ...
2999996,NPRW2300000001,NPRW2300000001.31919,머니투데이 2022년 기사,정현수|기자|,머니투데이,20220628,IT/과학,"IT_과학>모바일, IT_과학>콘텐츠, 지역>경기",NPRW2300000001.31919.6,이 과정에서 지문이나 얼굴 등 생체인증 정보를 추가로 등록할 수 있다. 신분확인의 ...,"모바일, IT_과학, 콘텐츠, 지역, 경기",이 과정에서 지문이나 얼굴 등 생체인증 정보를 추가로 등록할 수 있다. 신분확인의 ...
2999997,NPRW2300000001,NPRW2300000001.31919,머니투데이 2022년 기사,정현수|기자|,머니투데이,20220628,IT/과학,"IT_과학>모바일, IT_과학>콘텐츠, 지역>경기",NPRW2300000001.31919.7,모바일 주민등록증은 개인정보 보호 장치도 마련했다. 기본 화면에는 성명과 주소의 일...,"모바일, IT_과학, 콘텐츠, 지역, 경기",모바일 주민등록증은 개인정보 보호 장치도 마련했다. 기본 화면에는 성명과 주소의 일...
2999998,NPRW2300000001,NPRW2300000001.31919,머니투데이 2022년 기사,정현수|기자|,머니투데이,20220628,IT/과학,"IT_과학>모바일, IT_과학>콘텐츠, 지역>경기",NPRW2300000001.31919.8,한창섭 행안부 차관은 “디지털 신원증명으로써 주민등록증 모바일 확인서비스가 시행되면...,"모바일, IT_과학, 콘텐츠, 지역, 경기",한창섭 행안부 차관은 디지털 신원증명으로써 주민등록증 모바일 확인서비스가 시행되면...


In [6]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

# 기존의 소분류 라벨
small_label = ['국회_정당', '북한', '선거', '외교', '청와대', '행정_자치', '골프', '농구_배구', '야구_메이저리그',
               '야구_일본프로야구', '올림픽_아시안게임', '축구_월드컵', '축구_한국프로축구', '축구_해외축구',
               '교육_시험', '날씨', '노동_복지', '미디어', '사건_사고', '여성', '의료_건강', '장애인', '환경',
               '미술_건축', '방송_연예', '생활', '요리_여행', '음악', '전시_공연', '종교', '출판', '학술_문화재',
               '러시아', '미국_북미', '아시아', '유럽_EU', '일본', '중국', '중남미', '중동_아프리카', '국제경제',
               '금융_재테크', '무역', '반도체', '부동산', '산업_기업', '서비스_쇼핑', '외환', '유통', '자동차',
               '자원', '증권_증시', '취업_창업', '과학', '모바일', '보안', '인터넷_SNS', '콘텐츠']

# 라벨 매핑 딕셔너리
small_label2id = {label: i for i, label in enumerate(small_label)}
small_id2label = {i: label for label, i in small_label2id.items()}

# 다중 라벨 변환 함수
def extract_labels(label_str):
    """쉼표로 구분된 라벨을 리스트로 변환 (소분류 기준)"""
    labels = label_str.split(',')  # 쉼표로 구분하여 리스트로 변환
    labels = [label.strip() for label in labels if label.strip() in small_label2id]  # 소분류 라벨만 유지
    return labels if labels else None  # 유효한 라벨이 없으면 None 반환

# 라벨 리스트로 변환
merged_data['label_list'] = merged_data['NEWS_SML_SUBJ_CD'].apply(extract_labels)
merged_data

/tmp/ipykernel_1377225/3123018005.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['label_list'] = data['NEWS_SML_SUBJ_CD'].apply(extract_labels)


,file_id,doc_id,title,author,publisher,date,topic,original_topic,sentence_id,sentence,NEWS_SML_SUBJ_CD,NEWS_CNTS,label_list
0,NIRW2300000001,NIRW2300000001.1,노컷뉴스 2022년 기사,박중석 부산CBS 박중석 기자,노컷뉴스,20220101,정치,"지역>부산, 지역>경남, 지역>울산",NIRW2300000001.1.1,[영상]“위기를 이겨내고 일상으로” 부산 기관장 신년사,"부산, 지역, 경남, 지역, 울산",영상 위기를 이겨내고 일상으로 부산 기관장 신년사,None
1,NIRW2300000001,NIRW2300000001.1,노컷뉴스 2022년 기사,박중석 부산CBS 박중석 기자,노컷뉴스,20220101,정치,"지역>부산, 지역>경남, 지역>울산",NIRW2300000001.1.2,취임 2년 차를 맞는 박형준 부산시장은 코로나19 위기를 이겨내고 있는 부산시민들에...,"부산, 지역, 경남, 지역, 울산",취임 2년 차를 맞는 박형준 부산시장은 코로나19 위기를 이겨내고 있는 부산시민들에...,None
2,NIRW2300000001,NIRW2300000001.1,노컷뉴스 2022년 기사,박중석 부산CBS 박중석 기자,노컷뉴스,20220101,정치,"지역>부산, 지역>경남, 지역>울산",NIRW2300000001.1.3,"박 시장은 “코로나19는 번번이 우리의 희망의 길목을 막아섰고, 지금도 여전히 민생...","부산, 지역, 경남, 지역, 울산",박 시장은 코로나19는 번번이 우리의 희망의 길목을 막아섰고 지금도 여전히 민생...,None
3,NIRW2300000001,NIRW2300000001.1,노컷뉴스 2022년 기사,박중석 부산CBS 박중석 기자,노컷뉴스,20220101,정치,"지역>부산, 지역>경남, 지역>울산",NIRW2300000001.1.4,그러면서 “숱한 어려움을 헤쳐온 위대한 시민의 힘으로 2022년을 코로나를 극복하고...,"부산, 지역, 경남, 지역, 울산",그러면서 숱한 어려움을 헤쳐온 위대한 시민의 힘으로 2022년을 코로나를 극복하고...,None
4,NIRW2300000001,NIRW2300000001.1,노컷뉴스 2022년 기사,박중석 부산CBS 박중석 기자,노컷뉴스,20220101,정치,"지역>부산, 지역>경남, 지역>울산",NIRW2300000001.1.5,박 시장은 “치밀한 전략으로 2030세계박람회 유치를 위한 국제박람회기구 현지 실사...,"부산, 지역, 경남, 지역, 울산",박 시장은 치밀한 전략으로 2030세계박람회 유치를 위한 국제박람회기구 현지 실사...,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,NPRW2300000001,NPRW2300000001.31919,머니투데이 2022년 기사,정현수|기자|,머니투데이,20220628,IT/과학,"IT_과학>모바일, IT_과학>콘텐츠, 지역>경기",NPRW2300000001.31919.5,모바일 주민등록증을 이용하려면 본인 명의의 스마트폰으로 서비스 등록을 해야 한다. ...,"모바일, IT_과학, 콘텐츠, 지역, 경기",모바일 주민등록증을 이용하려면 본인 명의의 스마트폰으로 서비스 등록을 해야 한다. ...,"[모바일, 콘텐츠]"
2999996,NPRW2300000001,NPRW2300000001.31919,머니투데이 2022년 기사,정현수|기자|,머니투데이,20220628,IT/과학,"IT_과학>모바일, IT_과학>콘텐츠, 지역>경기",NPRW2300000001.31919.6,이 과정에서 지문이나 얼굴 등 생체인증 정보를 추가로 등록할 수 있다. 신분확인의 ...,"모바일, IT_과학, 콘텐츠, 지역, 경기",이 과정에서 지문이나 얼굴 등 생체인증 정보를 추가로 등록할 수 있다. 신분확인의 ...,"[모바일, 콘텐츠]"
2999997,NPRW2300000001,NPRW2300000001.31919,머니투데이 2022년 기사,정현수|기자|,머니투데이,20220628,IT/과학,"IT_과학>모바일, IT_과학>콘텐츠, 지역>경기",NPRW2300000001.31919.7,모바일 주민등록증은 개인정보 보호 장치도 마련했다. 기본 화면에는 성명과 주소의 일...,"모바일, IT_과학, 콘텐츠, 지역, 경기",모바일 주민등록증은 개인정보 보호 장치도 마련했다. 기본 화면에는 성명과 주소의 일...,"[모바일, 콘텐츠]"
2999998,NPRW2300000001,NPRW2300000001.31919,머니투데이 2022년 기사,정현수|기자|,머니투데이,20220628,IT/과학,"IT_과학>모바일, IT_과학>콘텐츠, 지역>경기",NPRW2300000001.31919.8,한창섭 행안부 차관은 “디지털 신원증명으로써 주민등록증 모바일 확인서비스가 시행되면...,"모바일, IT_과학, 콘텐츠, 지역, 경기",한창섭 행안부 차관은 디지털 신원증명으로써 주민등록증 모바일 확인서비스가 시행되면...,"[모바일, 콘텐츠]"


In [7]:
# 'label_list'가 None인 행 제거
data = merged_data.dropna(subset=['label_list'])

# 인덱스 리셋
data = data.reset_index(drop=True)
data

,file_id,doc_id,title,author,publisher,date,topic,original_topic,sentence_id,sentence,NEWS_SML_SUBJ_CD,NEWS_CNTS,label_list
0,NIRW2300000001,NIRW2300000001.2,노컷뉴스 2022년 기사,CBS노컷뉴스 임진수 기자 임진수,노컷뉴스,20220101,경제,"경제>금융_재테크, 경제>외환, 경제>부동산",NIRW2300000001.2.1,올해는 대출 한파 좀 풀릴까?…규제·금리·대선이 변수,"금융_재테크, 경제, 외환, 경제, 부동산",올해는 대출 한파 좀 풀릴까 규제 금리 대선이 변수,"[금융_재테크, 외환, 부동산]"
1,NIRW2300000001,NIRW2300000001.2,노컷뉴스 2022년 기사,CBS노컷뉴스 임진수 기자 임진수,노컷뉴스,20220101,경제,"경제>금융_재테크, 경제>외환, 경제>부동산",NIRW2300000001.2.2,지난해 하반기 가계부채 폭증 영향으로 사상 초유의 대출 중단 사태가 빚어지는 등 대...,"금융_재테크, 경제, 외환, 경제, 부동산",지난해 하반기 가계부채 폭증 영향으로 사상 초유의 대출 중단 사태가 빚어지는 등 대...,"[금융_재테크, 외환, 부동산]"
2,NIRW2300000001,NIRW2300000001.2,노컷뉴스 2022년 기사,CBS노컷뉴스 임진수 기자 임진수,노컷뉴스,20220101,경제,"경제>금융_재테크, 경제>외환, 경제>부동산",NIRW2300000001.2.3,"다만, 올해도 금융당국의 대출 총량관리가 이어지고, 가장 강력한 대출규제로 꼽히는 ...","금융_재테크, 경제, 외환, 경제, 부동산",다만 올해도 금융당국의 대출 총량관리가 이어지고 가장 강력한 대출규제로 꼽히는 ...,"[금융_재테크, 외환, 부동산]"
3,NIRW2300000001,NIRW2300000001.2,노컷뉴스 2022년 기사,CBS노컷뉴스 임진수 기자 임진수,노컷뉴스,20220101,경제,"경제>금융_재테크, 경제>외환, 경제>부동산",NIRW2300000001.2.4,지난해 8월말 NH농협은행을 필두로 시작된 대출 중단사태가 연말·연시에 접어들면서 ...,"금융_재테크, 경제, 외환, 경제, 부동산",지난해 8월말 NH농협은행을 필두로 시작된 대출 중단사태가 연말 연시에 접어들면서 ...,"[금융_재테크, 외환, 부동산]"
4,NIRW2300000001,NIRW2300000001.2,노컷뉴스 2022년 기사,CBS노컷뉴스 임진수 기자 임진수,노컷뉴스,20220101,경제,"경제>금융_재테크, 경제>외환, 경제>부동산",NIRW2300000001.2.5,"NH농협은행은 물론이고 KB국민은행과 신한은행, 우리은행, 하나은행 등 지점별 총량...","금융_재테크, 경제, 외환, 경제, 부동산",NH농협은행은 물론이고 KB국민은행과 신한은행 우리은행 하나은행 등 지점별 총량...,"[금융_재테크, 외환, 부동산]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018448,NPRW2300000001,NPRW2300000001.31919,머니투데이 2022년 기사,정현수|기자|,머니투데이,20220628,IT/과학,"IT_과학>모바일, IT_과학>콘텐츠, 지역>경기",NPRW2300000001.31919.5,모바일 주민등록증을 이용하려면 본인 명의의 스마트폰으로 서비스 등록을 해야 한다. ...,"모바일, IT_과학, 콘텐츠, 지역, 경기",모바일 주민등록증을 이용하려면 본인 명의의 스마트폰으로 서비스 등록을 해야 한다. ...,"[모바일, 콘텐츠]"
2018449,NPRW2300000001,NPRW2300000001.31919,머니투데이 2022년 기사,정현수|기자|,머니투데이,20220628,IT/과학,"IT_과학>모바일, IT_과학>콘텐츠, 지역>경기",NPRW2300000001.31919.6,이 과정에서 지문이나 얼굴 등 생체인증 정보를 추가로 등록할 수 있다. 신분확인의 ...,"모바일, IT_과학, 콘텐츠, 지역, 경기",이 과정에서 지문이나 얼굴 등 생체인증 정보를 추가로 등록할 수 있다. 신분확인의 ...,"[모바일, 콘텐츠]"
2018450,NPRW2300000001,NPRW2300000001.31919,머니투데이 2022년 기사,정현수|기자|,머니투데이,20220628,IT/과학,"IT_과학>모바일, IT_과학>콘텐츠, 지역>경기",NPRW2300000001.31919.7,모바일 주민등록증은 개인정보 보호 장치도 마련했다. 기본 화면에는 성명과 주소의 일...,"모바일, IT_과학, 콘텐츠, 지역, 경기",모바일 주민등록증은 개인정보 보호 장치도 마련했다. 기본 화면에는 성명과 주소의 일...,"[모바일, 콘텐츠]"
2018451,NPRW2300000001,NPRW2300000001.31919,머니투데이 2022년 기사,정현수|기자|,머니투데이,20220628,IT/과학,"IT_과학>모바일, IT_과학>콘텐츠, 지역>경기",NPRW2300000001.31919.8,한창섭 행안부 차관은 “디지털 신원증명으로써 주민등록증 모바일 확인서비스가 시행되면...,"모바일, IT_과학, 콘텐츠, 지역, 경기",한창섭 행안부 차관은 디지털 신원증명으로써 주민등록증 모바일 확인서비스가 시행되면...,"[모바일, 콘텐츠]"


In [10]:
filtered_data = data[data['label_list'].apply(lambda x: len(x) == 1)]
filtered_data = filtered_data.reset_index(drop=True)
filtered_data.to_csv('/home/lhch9550/공모전/KPF-BERT-CLS/processed_data.csv')

In [17]:
# MultiLabel Binarizer 적용 (라벨이 없는 경우도 포함)
mlb = MultiLabelBinarizer(classes=small_label)
label_matrix = mlb.fit_transform(data['label_list'])

# 새로운 데이터셋 형식으로 변환 (인덱스 유지)
label_df = pd.DataFrame(label_matrix, columns=mlb.classes_, index=data.index)
label_df

,국회_정당,북한,선거,외교,청와대,행정_자치,골프,농구_배구,야구_메이저리그,야구_일본프로야구,...,유통,자동차,자원,증권_증시,취업_창업,과학,모바일,보안,인터넷_SNS,콘텐츠
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018448,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
2018449,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
2018450,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
2018451,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1


In [18]:
data = data.join(label_df)
data

,file_id,doc_id,title,author,publisher,date,topic,original_topic,sentence_id,sentence,...,유통,자동차,자원,증권_증시,취업_창업,과학,모바일,보안,인터넷_SNS,콘텐츠
0,NIRW2300000001,NIRW2300000001.2,노컷뉴스 2022년 기사,CBS노컷뉴스 임진수 기자 임진수,노컷뉴스,20220101,경제,"경제>금융_재테크, 경제>외환, 경제>부동산",NIRW2300000001.2.1,올해는 대출 한파 좀 풀릴까?…규제·금리·대선이 변수,...,0,0,0,0,0,0,0,0,0,0
1,NIRW2300000001,NIRW2300000001.2,노컷뉴스 2022년 기사,CBS노컷뉴스 임진수 기자 임진수,노컷뉴스,20220101,경제,"경제>금융_재테크, 경제>외환, 경제>부동산",NIRW2300000001.2.2,지난해 하반기 가계부채 폭증 영향으로 사상 초유의 대출 중단 사태가 빚어지는 등 대...,...,0,0,0,0,0,0,0,0,0,0
2,NIRW2300000001,NIRW2300000001.2,노컷뉴스 2022년 기사,CBS노컷뉴스 임진수 기자 임진수,노컷뉴스,20220101,경제,"경제>금융_재테크, 경제>외환, 경제>부동산",NIRW2300000001.2.3,"다만, 올해도 금융당국의 대출 총량관리가 이어지고, 가장 강력한 대출규제로 꼽히는 ...",...,0,0,0,0,0,0,0,0,0,0
3,NIRW2300000001,NIRW2300000001.2,노컷뉴스 2022년 기사,CBS노컷뉴스 임진수 기자 임진수,노컷뉴스,20220101,경제,"경제>금융_재테크, 경제>외환, 경제>부동산",NIRW2300000001.2.4,지난해 8월말 NH농협은행을 필두로 시작된 대출 중단사태가 연말·연시에 접어들면서 ...,...,0,0,0,0,0,0,0,0,0,0
4,NIRW2300000001,NIRW2300000001.2,노컷뉴스 2022년 기사,CBS노컷뉴스 임진수 기자 임진수,노컷뉴스,20220101,경제,"경제>금융_재테크, 경제>외환, 경제>부동산",NIRW2300000001.2.5,"NH농협은행은 물론이고 KB국민은행과 신한은행, 우리은행, 하나은행 등 지점별 총량...",...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018448,NPRW2300000001,NPRW2300000001.31919,머니투데이 2022년 기사,정현수|기자|,머니투데이,20220628,IT/과학,"IT_과학>모바일, IT_과학>콘텐츠, 지역>경기",NPRW2300000001.31919.5,모바일 주민등록증을 이용하려면 본인 명의의 스마트폰으로 서비스 등록을 해야 한다. ...,...,0,0,0,0,0,0,1,0,0,1
2018449,NPRW2300000001,NPRW2300000001.31919,머니투데이 2022년 기사,정현수|기자|,머니투데이,20220628,IT/과학,"IT_과학>모바일, IT_과학>콘텐츠, 지역>경기",NPRW2300000001.31919.6,이 과정에서 지문이나 얼굴 등 생체인증 정보를 추가로 등록할 수 있다. 신분확인의 ...,...,0,0,0,0,0,0,1,0,0,1
2018450,NPRW2300000001,NPRW2300000001.31919,머니투데이 2022년 기사,정현수|기자|,머니투데이,20220628,IT/과학,"IT_과학>모바일, IT_과학>콘텐츠, 지역>경기",NPRW2300000001.31919.7,모바일 주민등록증은 개인정보 보호 장치도 마련했다. 기본 화면에는 성명과 주소의 일...,...,0,0,0,0,0,0,1,0,0,1
2018451,NPRW2300000001,NPRW2300000001.31919,머니투데이 2022년 기사,정현수|기자|,머니투데이,20220628,IT/과학,"IT_과학>모바일, IT_과학>콘텐츠, 지역>경기",NPRW2300000001.31919.8,한창섭 행안부 차관은 “디지털 신원증명으로써 주민등록증 모바일 확인서비스가 시행되면...,...,0,0,0,0,0,0,1,0,0,1


In [19]:
# 불필요한 컬럼 제거 ('NEWS_SML_SUBJ_CD', 'label_list' 제거)
columns_to_keep = ['NEWS_CNTS'] + small_label
data = data[columns_to_keep]

In [22]:
# 라벨 컬럼 중 하나라도 1인 행 확인
label_exists = data[small_label].sum(axis=1) > 0

# 라벨이 하나라도 있는 행의 개수 출력
num_label_rows = label_exists.sum()
print(f"라벨이 하나라도 포함된 기사 개수: {num_label_rows}")

# 라벨이 없는 행 개수 출력
num_no_label_rows = (~label_exists).sum()
print(f"라벨이 없는 기사 개수: {num_no_label_rows}")

라벨이 하나라도 포함된 기사 개수: 2018453
라벨이 없는 기사 개수: 0


In [23]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd

# 데이터 샘플링 함수
def sampling_func(data, sample_pct, _seed):
    np.random.seed(_seed)
    sample_n = int(len(data) * sample_pct)
    return data.sample(n=sample_n, random_state=_seed)  # pandas sample 사용

# ClsDataset 클래스 (멀티 라벨 대응)
class ClsDataset(torch.utils.data.Dataset):  # torch.utils.data.Dataset을 명확히 지정
    def __init__(self, data, label_columns):
        self.data = data
        self.texts = data['NEWS_CNTS'].tolist()  # 텍스트 데이터
        self.labels = data[label_columns].values  # MultiLabel One-Hot Encoding 데이터

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = torch.tensor(self.labels[idx], dtype=torch.float32)  # 다중 라벨 변환
        return text, label

# collate_fn (배치 처리를 위한 함수)
def collate_fn(batch):
    texts, labels = zip(*batch)
    return list(texts), torch.stack(labels)

# 수정된 load_dataset (멀티 라벨 처리 포함)
from torch.utils.data import DataLoader

def load_dataset(data, con):
    """멀티 라벨 분류를 위한 데이터셋 로딩 함수"""
    
    # NaN 값 제거 (라벨 없는 데이터 제거)
    data = data.dropna(subset=small_label)  # small_label에 해당하는 컬럼이 NaN이면 제거
    
    # train/valid/test 데이터 비율
    train_pct, valid_pct, test_pct = 0.7, 0.2, 0.1  

    # 데이터 샘플링
    train_data = data.sample(frac=train_pct, random_state=con.seed)
    remaining_data = data.drop(train_data.index)
    valid_data = remaining_data.sample(frac=valid_pct / (valid_pct + test_pct), random_state=con.seed)
    test_data = remaining_data.drop(valid_data.index)

    # DataLoader 생성
    train_dataset = ClsDataset(train_data, label_columns=small_label)
    valid_dataset = ClsDataset(valid_data, label_columns=small_label)
    test_dataset = ClsDataset(test_data, label_columns=small_label)

    train_dataloader = DataLoader(train_dataset, batch_size=con.batch_size, shuffle=True)
    valid_dataloader = DataLoader(valid_dataset, batch_size=con.batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=con.batch_size, shuffle=False)

    print(f"Train Data: {len(train_data)} | Valid Data: {len(valid_data)} | Test Data: {len(test_data)}")

    return train_dataloader, valid_dataloader, test_dataloader

In [24]:
from torch.utils.data import DataLoader
import torch
from transformers import BertConfig, BertForSequenceClassification, AutoTokenizer, AdamW
import torch.nn.functional as F
from sklearn.metrics import classification_report, f1_score
from tqdm import tqdm

# Config 클래스를 통해 설정값을 가져옵니다.
from config import Config
_config = Config()
_config.num_labels = len(small_label2id)  # 라벨의 고유 개수를 설정

# 모델 초기화 함수 (멀티 라벨 분류에 맞게 수정)
def initialize_model(con):
    model = BertForSequenceClassification.from_pretrained(
        con.model_name, 
        num_labels=con.num_labels,  # 다중 라벨 개수 설정
        problem_type="multi_label_classification",  # 멀티 라벨 문제 지정
        ignore_mismatched_sizes=True
    )
    model.to(con.device)  # GPU 이동
    tokenizer = AutoTokenizer.from_pretrained(con.model_name)
    return model, tokenizer

# 옵티마이저 설정
def set_optimizer(model, con):
    optimizer_grouped_parameters = [
        {'params': model.bert.parameters(), 'lr': 3e-5},
        {'params': model.classifier.parameters(), 'lr': con.learning_rate}
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=con.learning_rate, eps=con.adam_epsilon, no_deprecation_warning=True)
    return optimizer

# 멀티 라벨 학습 에포크
def train_epoch(epoch, model, dataloader, optimizer, tokenizer, con):
    model.train()
    total_loss = 0.0
    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1} Training", unit="batch")

    criterion = torch.nn.BCEWithLogitsLoss()  # BCE Loss 적용 (멀티 라벨 분류)

    for batch in progress_bar:
        model.zero_grad()
        
        text_inputs = [str(text) if isinstance(text, (str, int, float)) else "" for text in batch[0]]

        sample = tokenizer(
            text_inputs, 
            padding='max_length', 
            truncation=True, 
            max_length=con.max_seq_len, 
            return_tensors="pt"
        )['input_ids'].to(con.device)

        labels = batch[1].float().to(con.device)  # 멀티 라벨 (float 타입 필요)

        outputs = model(sample)
        logits = outputs.logits  # 예측값
        loss = criterion(logits, labels)  # BCE Loss 계산

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), con.max_grad_norm)
        optimizer.step()
        total_loss += loss.item()

        progress_bar.set_postfix(loss=loss.item())

    return total_loss / len(dataloader)

# 검증 에포크
def valid_epoch(epoch, dataloader, model, tokenizer, con, id2label):
    model.eval()
    total_loss = 0.0
    all_token_predictions = []
    all_token_labels = []
    
    criterion = torch.nn.BCEWithLogitsLoss()  # BCE Loss 적용
    tepoch = tqdm(dataloader, unit="batch", leave=False)

    with torch.no_grad():
        for batch in tepoch:
            sample = tokenizer(batch[0], padding='max_length', truncation=True,
                               max_length=con.max_seq_len, return_tensors="pt")['input_ids'].to(con.device)

            labels = batch[1].float().to(con.device)  # 멀티 라벨 (float 타입 필요)

            outputs = model(sample)
            logits = outputs.logits
            loss = criterion(logits, labels)

            total_loss += loss.item()

            # 다중 라벨 예측 (0.5 이상일 경우 해당 클래스로 예측)
            preds = (torch.sigmoid(logits) > 0.5).int().cpu().numpy()
            all_token_labels.extend(labels.cpu().numpy())
            all_token_predictions.extend(preds)

            tepoch.set_postfix(loss=loss.mean().item())

    # F1 Score 계산 (micro 평균)
    token_f1 = f1_score(all_token_labels, all_token_predictions, average="micro")
    print(f'[Epoch {epoch+1}] -> F1 Score: {token_f1:.4f}, Validation Loss: {total_loss / len(dataloader):.4f}')
    
    return total_loss / len(dataloader), token_f1

def test_epoch(dataloader, model, tokenizer, con, id2label, evaluate=False):
    model.eval()
    total_loss = 0.0
    all_token_predictions = []
    all_token_labels = []
    
    criterion = torch.nn.BCEWithLogitsLoss()
    tepoch = tqdm(dataloader, unit="batch", leave=False)

    with torch.no_grad():
        for batch in tepoch:
            sample = tokenizer(batch[0], padding='max_length', truncation=True,
                               max_length=con.max_seq_len, return_tensors="pt")['input_ids'].to(con.device)

            labels = batch[1].float().to(con.device)

            outputs = model(sample)
            logits = outputs.logits
            loss = criterion(logits, labels)

            total_loss += loss.item()

            # 다중 라벨 예측 (0.5 이상일 경우 해당 클래스로 예측)
            preds = (torch.sigmoid(logits) > 0.5).int().cpu().numpy()
            all_token_labels.extend(labels.cpu().numpy())
            all_token_predictions.extend(preds)

            tepoch.set_postfix(loss=loss.mean().item())

    # 라벨 ID → 라벨명 변환 (seqeval을 위해)
    true_labels = [[id2label[i] for i, v in enumerate(row) if v == 1] for row in all_token_labels]
    pred_labels = [[id2label[i] for i, v in enumerate(row) if v == 1] for row in all_token_predictions]

    # MultiLabelBinarizer 적용 (sklearn 변환)
    mlb = MultiLabelBinarizer(classes=list(id2label.values()))
    true_binary = mlb.fit_transform(true_labels)
    pred_binary = mlb.transform(pred_labels)

    # 평가 지표 계산 (seqeval 대신 sklearn f1_score 사용)
    f1 = f1_score(true_binary, pred_binary, average='macro')

    if evaluate:
        print("[Test Dataset Evaluation Metrics]\n")
        print("[Test Classification Report]\n")
        print(classification_report(true_binary, pred_binary, target_names=list(id2label.values())))

    return total_loss / len(dataloader), f1  # 평균 Loss 및 F1-score 반환

def train(model, tokenizer, train_dataloader, valid_dataloader, test_dataloader, optimizer, con, id2label):
    for epoch in range(con.epoch):
        print(f"Epoch {epoch+1}/{con.epoch}")
        
        # 학습
        train_loss = train_epoch(epoch, model, train_dataloader, optimizer, tokenizer, con)

        # 검증
        valid_loss, valid_f1 = valid_epoch(epoch, valid_dataloader, model, tokenizer, con, id2label)

        # 테스트
        test_loss, test_f1 = test_epoch(test_dataloader, model, tokenizer, con, id2label)

        print(f"Epoch {epoch+1} Summary:")
        print(f"Train Loss: {train_loss:.4f}, Valid Loss: {valid_loss:.4f}, Test Loss: {test_loss:.4f}")
        print(f"Valid F1: {valid_f1:.4f}, Test F1: {test_f1:.4f}\n")

In [25]:
# 모델 초기화 (멀티 라벨 분류 지원)
model, tokenizer = initialize_model(_config)  
optimizer = set_optimizer(model, _config)

# 데이터셋 준비
train_dataloader, valid_dataloader, test_dataloader = load_dataset(data, _config)

# 학습 시작 (멀티 라벨 학습 진행)
train(model, tokenizer, train_dataloader, valid_dataloader, test_dataloader, optimizer, _config, small_id2label)

/home/lhch9550/anaconda3/lib/python3.8/site-packages/transformers/modeling_utils.py:484: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map

Train Data: 1412917 | Valid Data: 403691 | Test Data: 201845
Epoch 1/2


Epoch 1 Training: 100%|██████████| 88308/88308 [4:34:06<00:00,  5.37batch/s, loss=0.0558]  


[Epoch 1] -> F1 Score: 0.6848, Validation Loss: 0.0526


/home/lhch9550/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1760: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


Epoch 1 Summary:
Train Loss: 0.0603, Valid Loss: 0.0526, Test Loss: 0.0524
Valid F1: 0.6848, Test F1: 0.5441

Epoch 2/2


Epoch 2 Training: 100%|██████████| 88308/88308 [4:34:09<00:00,  5.37batch/s, loss=0.041]   


[Epoch 2] -> F1 Score: 0.7050, Validation Loss: 0.0502


/home/lhch9550/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1760: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


Epoch 2 Summary:
Train Loss: 0.0503, Valid Loss: 0.0502, Test Loss: 0.0501
Valid F1: 0.7050, Test F1: 0.5766



In [26]:
import torch
import pandas as pd
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification, AutoTokenizer

# Test 데이터 예측 함수
def predict(model, tokenizer, test_dataloader, con, small_id2label):
    model.eval()  # 평가 모드
    predictions = []
    true_labels = []
    texts = []

    with torch.no_grad():
        for batch in tqdm(test_dataloader, desc="Predicting"):
            text_inputs = [str(text) for text in batch[0]]  # 입력 텍스트
            encoded_inputs = tokenizer(
                text_inputs, 
                padding=True, 
                truncation=True, 
                max_length=con.max_seq_len, 
                return_tensors="pt"
            )
            encoded_inputs = {k: v.to(con.device) for k, v in encoded_inputs.items()}
            
            outputs = model(**encoded_inputs)
            logits = outputs.logits  # 예측값
            preds = torch.sigmoid(logits).cpu().numpy()  # 멀티라벨 예측값 확률

            # 임계값 0.5 이상이면 1 (해당 라벨 존재), 아니면 0
            preds = (preds > 0.5).astype(int)

            predictions.extend(preds.tolist())
            true_labels.extend(batch[1].cpu().numpy())  # 실제 정답
            texts.extend(batch[0])  # 원본 텍스트 저장

    # 라벨 ID → 라벨명 변환 (멀티라벨이므로 리스트로 변환)
    pred_labels = [[small_id2label[idx] for idx, val in enumerate(p) if val == 1] for p in predictions]
    true_labels = [[small_id2label[idx] for idx, val in enumerate(t) if val == 1] for t in true_labels]

    # 결과 데이터프레임 생성
    df_results = pd.DataFrame({"Text": texts, "True Labels": true_labels, "Predicted Labels": pred_labels})

    return df_results

# `test_dataloader`로 추론 수행
df_results = predict(model, tokenizer, test_dataloader, _config, small_id2label)
df_results

Predicting: 100%|██████████| 12616/12616 [05:34<00:00, 37.76it/s]


,Text,True Labels,Predicted Labels
0,여기다 차주별 DSR 강화로 대출 한도가 큰 폭으로 줄어들 수 있다. 올해부터 총 ...,"[금융_재테크, 부동산, 외환]","[금융_재테크, 부동산]"
1,드라마 펜트하우스 시즌 3의 김소연 41 이 2021 SBS 연기대상에서 대상을...,"[방송_연예, 출판]",[방송_연예]
2,베스트 커플상은 홍천기 의 안효섭과 김유정이 차지했고 그 해 우리는 의 최우식과...,"[방송_연예, 출판]","[미술_건축, 전시_공연, 출판]"
3,이 중 38 인 8명은 도내 또는 다른 지역 확진자의 접촉자 8명 38 은 감염...,[취업_창업],[의료_건강]
4,그러나 인구 기준 10만 명당 으로 보면 통영이 9.9명으로 확산세가 가장 거세고...,[취업_창업],[의료_건강]
...,...,...,...
201840,모빌리티 매각 정해진 바 없다 노조 달래기 나선 카카오,"[노동_복지, 미디어]","[노동_복지, 미디어]"
201841,삼성전자는 28일 오전 서울시 중구 태평로빌딩 기자실에서 에어컨 혁신 기술 브리핑...,"[서비스_쇼핑, 유통]","[서비스_쇼핑, 유통, 모바일]"
201842,그러면서 무풍에어컨이 첫 개발 이후로도 무풍 패널 서큘레이터 급속 냉방 고효율...,"[서비스_쇼핑, 유통]","[서비스_쇼핑, 유통, 모바일]"
201843,한편 삼성은 이날 무풍에어컨이 지속적인 인기를 받은 결과로 현재까지 국내에서 스탠드...,"[서비스_쇼핑, 유통]","[서비스_쇼핑, 유통]"


In [46]:
filtered_data = df_results[df_results['True Labels'].apply(lambda x: '교육_시험' in x)]
filtered_data = filtered_data[filtered_data['Predicted Labels'].apply(lambda x: '교육_시험' in x)]
filtered_data

,Text,True Labels,Predicted Labels
63,기초학력 프로그램과 초급 영어학습반을 보강 운영하고 재교육을 원하는 일반 성인까지 ...,[교육_시험],[교육_시험]
64,관악구에 따르면 샘 SNU Active Mentoring SAM 멘토링은 지역...,[교육_시험],[교육_시험]
123,이밖에도 체험중심의 학생 안전교육이 강화되는데 방사능 교육용 가상현실 기기를 희망하...,[교육_시험],[교육_시험]
204,2022학년도 부산지역 대입 정시모집 원서접수가 3일 마감된 가운데 상위권 대학은 ...,[교육_시험],[교육_시험]
205,인문 사회계열에서 노어노문학과가 7.06 대 1로 최고 경쟁률을 기록했고 자연계열...,[교육_시험],[교육_시험]
...,...,...,...
201589,그러면서 A씨가 교단에 복귀한다고 할 때 이 모습을 교육 현장에서 마주하게 될 ...,"[교육_시험, 미디어, 여성]","[교육_시험, 노동_복지, 미디어]"
201755,서울대는 해당 논문의 공저자 중 서울대 소속으로 표기된 모든 연구자를 대상으로 조사...,[교육_시험],"[교육_시험, 미디어]"
201802,교육급여 학습특별지원금은 코로나19 장기화로 인해 초래된 저소득층 학생의 학습격차를...,[교육_시험],[교육_시험]
201803,지원 방식은 신용 체크 카드 포인트 EBS 맞춤형 쿠폰 간편결제 포인트 중 선택...,[교육_시험],[교육_시험]


In [ ]:
def save_model(model, tokenizer, save_path):
    model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)
    print(f"Model saved at: {save_path}")

# 모델 저장 경로
save_path = ""

# 모델 저장 실행
save_model(model, tokenizer, save_path)